In [1]:
import matplotlib as mpl
# mpl.use('Agg')
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42
mpl.rcParams['text.usetex'] = False
mpl.rcParams['font.sans-serif'] = 'Arial'
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['image.interpolation'] = 'none'

import os, re
from pathlib import Path
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pprint import pprint
import scipy
import seaborn as sns

from Bio import SeqIO
from scipy.interpolate import UnivariateSpline
import matplotlib.transforms as mtransforms
from matplotlib.patches import FancyBboxPatch

import metapredict as meta

%matplotlib inline

C:\Users\13kov\anaconda3\envs\metapredict_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
fasta_path = Path('uniprot-reviewed_yes+AND+proteome_up000005640.fasta')
output_path = Path('/Users/13kov/Jupyter code/TF-RNA/ARM_search')

if not output_path.is_dir():
    output_path.mkdir()

In [3]:
def calc_tat_corr(seq): 
    # For correlation task, the TAT ARM sequence is: 49-RKKRRQRRR-57 and there may be two critical R’s, R52 and R53. R53 seems to be the most critical.
    # I would suggest creating a kernel that looks something like:
    # [111RR0111] - where the non-critical R/K’s are treated as generic + charges, the critical R’s have to be R’s, and the Q is a neutral amino acid
    # and also try: [1111R0111] - where only R53 is treated as critical

    # Define kernel
    # 1 represents a positive AA, 0  represents a neutral AA
    ker=[1, 1, 1, 1, 1, 0, 1, 1, 1] # For Q=0
#     ker=[1, 1, 1, 1, 1, 1, 1, 1, 1] # For Q=1
    kmer = len(ker)
    
    # For filtering only on disordered regions
    disorder_scores, disorder_regions = calc_disorder(seq)
    disorder_threshold = 0.2 # Value of 0.2 suggested by Salman
    
    corr_vect = np.zeros(shape=len(seq), )
    subseq_vect = strs = ['']*len(seq)
    
    for idx, a in enumerate(seq):
        # Check for R's in disordered regions
        if a == 'R' and idx >= 4 and idx <= len(seq)-4 and disorder_scores[idx] >= disorder_threshold:
            sub_seq = seq[idx-4:idx+5] # Get 9-mer subsequence corresponding to the window around a disordered R
            
            # Digitize the subsequence based on charge to get a signal
            sig=np.zeros(len(ker)).astype(int)
            for j, val in enumerate(sub_seq):
                if (val=='R' or val=='K'):
                    sig[j]=1 # Any positive residue becomes a 1
                else:
                    sig[j]=0 # Any neutral or negative residue becomes a 0
           
            # Cross-correlation of the signal with the kernel
            n0=scipy.signal.correlate(sig, ker, mode='full', method='direct')[8] # Compute the x-corr at 0 lag (no shift)
            corr_score=n0/np.sum(ker) # After normalization the max score will be 8 because Q is 0
#             print(sub_seq)
#             print(sig)
#             print('Correlation score = '+str(corr_score))
            corr_vect[idx] = corr_score
            subseq_vect[idx] = sub_seq
                
    return corr_vect, subseq_vect

In [4]:
def calc_disorder(seq):
    predictor = meta.predict_disorder_domains(seq)
    
    score = predictor.disorder
    regions = predictor.disordered_domain_boundaries
    
    return score, regions

In [5]:
# TF_df = pd.read_csv('TF_list.csv',header=None).T
# TF_query_list=TF_df.loc[0].tolist()
# TF_full_df = pd.DataFrame()

# TF_output = {} # Declare dictionary

# for query in TF_query_list:
# #     print(query)
#     count = 0
#     with open(fasta_path, 'r') as file:
#         for record in SeqIO.parse(file, "fasta"):
#             description = record.description
# #             print(description)
#             gene_name = description[(description.find("GN=")+3):description.find(" PE")]
# #             print(gene_name)
#             if query == gene_name:
# #                 print(f'Found {query} in {record.id}')  # This helps to see if there are multiple records for the same gene
#                 ident = record.id
#                 seq = str(record.seq).replace('U', 'C') # Replace U (selenocysteine) with C (cysteine)
#                 count += 1
                
#                 # Make sure this if statement is one level down from the above
#                 if count > 1:
#                     print(f'Found {count} entries for {query}')
#                 elif count < 1:
#                 #         raise Exception(f'Could not find entry for {query} in fasta file')
#                     print(f'Could not find entry for {query} in fasta file')
#                 else:
#                     # TAT correlation
#                     corr_vect, subseq_vect = calc_tat_corr(seq)
#                     max_corr=max(corr_vect)
# #                     print('Max corr = ' + str(max_corr))
#                     TF_output[gene_name]=max_corr # Add max cross-correlation to dictionary
                    
#                     # Remove empty elements
#                     subseq_vect=np.array(subseq_vect).transpose()
#                     a=corr_vect>0
#                     corr_vect=corr_vect[a]
#                     subseq_vect=subseq_vect[a]
                    
#                     # Add to the main dataframe
#                     temp_df=pd.DataFrame(data=[[gene_name]*len(corr_vect),corr_vect,subseq_vect]).T
#                     TF_full_df= pd.concat([TF_full_df,temp_df])

# # Maximum xcorr
# TF_maxcorr_df=pd.DataFrame.from_dict(TF_output,orient='index',columns=['max_corr'])
# TF_maxcorr_df.to_csv('TF_maxcorr.csv')
# print('TF max correlation file printed')

# # All xcorr
# TF_full_df.columns =['Gene_name', 'Cross_corr', 'Subsequence']
# TF_full_df=TF_full_df[TF_full_df['Cross_corr'] > 0.25]  # Define threshold for filtering hits
# TF_full_df.to_csv('TF_full.csv',index=False)
# print('TF full sequence file printed')

In [6]:
# RBP_df = pd.read_csv('RBP_list.csv',header=None).T
# RBP_query_list=RBP_df.loc[0].tolist()
# RBP_full_df = pd.DataFrame()

# RBP_output = {} # Declare dictionary

# for query in RBP_query_list:
# #     print(query)
#     count = 0
#     with open(fasta_path, 'r') as file:
#         for record in SeqIO.parse(file, "fasta"):
#             description = record.description
# #             print(description)
#             gene_name = description[(description.find("GN=")+3):description.find(" PE")]
# #             print(gene_name)
#             if query == gene_name:
# #                 print(f'Found {query} in {record.id}')  # This helps to see if there are multiple records for the same gene
#                 ident = record.id
#                 seq = str(record.seq).replace('U', 'C') # Replace U (selenocysteine) with C (cysteine)
#                 count += 1
                
#                 # Make sure this if statement is one level down from the above
#                 if count > 1:
#                     print(f'Found {count} entries for {query}')
#                 elif count < 1:
#                 #         raise Exception(f'Could not find entry for {query} in fasta file')
#                     print(f'Could not find entry for {query} in fasta file')
#                 else:
#                     # TAT correlation
#                     corr_vect, subseq_vect = calc_tat_corr(seq)
#                     max_corr=max(corr_vect)
# #                     print('Max corr = ' + str(max_corr))
#                     RBP_output[gene_name]=max_corr
    
#                     # Remove empty elements
#                     subseq_vect=np.array(subseq_vect).transpose()
#                     a=corr_vect>0
#                     corr_vect=corr_vect[a]
#                     subseq_vect=subseq_vect[a]
                    
#                     # Add to the main dataframe
#                     temp_df=pd.DataFrame(data=[[gene_name]*len(corr_vect),corr_vect,subseq_vect]).T
#                     RBP_full_df= pd.concat([RBP_full_df,temp_df])

# # Maximum xcorr
# RBP_maxcorr_df=pd.DataFrame.from_dict(RBP_output,orient='index',columns=['max_corr'])
# RBP_maxcorr_df.to_csv('RBP_maxcorr.csv')
# print('RBP max correlation file printed')

# # All xcorr
# RBP_full_df.columns =['Gene_name', 'Cross_corr', 'Subsequence']
# RBP_full_df=RBP_full_df[RBP_full_df['Cross_corr'] > 0.25]  # Define threshold for filtering hits
# RBP_full_df.to_csv('RBP_full.csv',index=False)
# print('RBP full sequence file printed')

In [7]:
proteome_full_df = pd.DataFrame() # Declare full dataframe
proteome_output = {} # Declare dictionary for maxcorr

with open(fasta_path, 'r') as file:
    # Get proteome gene list from fasta file
    for record in SeqIO.parse(file, "fasta"):
        description = record.description
        gene_name = description[(description.find("GN=")+3):description.find(" PE")]
        seq = str(record.seq).replace('U', 'C') # Replace U (selenocysteine) with C (cysteine)
          
    # To do: Include a column for the uniprot ID in the record.id  
        
        try:
            corr_vect, subseq_vect  = calc_tat_corr(seq)
            max_corr=max(corr_vect)
            proteome_output[gene_name] = max_corr
            
            # Remove empty elements
            subseq_vect=np.array(subseq_vect).transpose()
            a=corr_vect>0
            corr_vect=corr_vect[a]
            subseq_vect=subseq_vect[a]

            # Add to the main dataframe
            temp_df=pd.DataFrame(data=[[gene_name]*len(corr_vect),corr_vect,subseq_vect]).T
            proteome_full_df= pd.concat([proteome_full_df,temp_df])
            
        except:
            print(f'Error thrown by {gene_name}')

# Maximum xcorr
proteome_maxcorr_df=pd.DataFrame.from_dict(proteome_output,orient='index',columns=['max_corr'])
proteome_maxcorr_df.to_csv('wholeproteome_maxcorr.csv')
print('Proteome file printed')

# All xcorr
proteome_full_df.columns =['Gene_name', 'Cross_corr', 'Subsequence']
proteome_full_df=proteome_full_df[proteome_full_df['Cross_corr'] > 0.25]  # Define threshold for filtering hits
proteome_full_df.to_csv('proteome_full.csv',index=False)
print('Proteome full sequence file printed')

Proteome file printed
Proteome full sequence file printed


In [8]:
# prot_df = pd.read_csv('wholeproteome_maxcorr.csv',index_col=0)
# RBP_df = pd.read_csv('RBP_maxcorr.csv',index_col=0)
# TF_df = pd.read_csv('TF_maxcorr.csv',index_col=0)

# # Filter TFs out of the whole proteome df
# prot_index = prot_df.index
# TF_index = TF_df.index
# mask1 = ~prot_index.isin(TF_index)
# prot_filt = prot_df.loc[mask1]

# # Filter TFs out of the RBP df
# RBP_index = RBP_df.index
# mask2 = ~RBP_index.isin(TF_index)
# RBP_filt = RBP_df.loc[mask2]

# prot_filt.to_csv('proteome_filt.csv')
# RBP_filt.to_csv('RBP_filt.csv')

# # Make figures
# fig=plt.figure(figsize = (8,10))
# ax = plt.axes()
# im=sns.violinplot(data = [TF_df['max_corr'].values,RBP_filt['max_corr'].values,prot_filt['max_corr'].values],palette="light:#4CB391", scale='width', ax=ax)
# im.set_xticklabels(['TFs','RBPs (-TFs)','Proteome (-TFs)'],fontsize=15)
# im.set_ylabel("Max Cross-Correlation", fontsize = 15)
# ax.grid(False)
# ax.set_facecolor('white')
# ax.patch.set_edgecolor('black')  
# ax.patch.set_linewidth('2') 
# ax.figure.savefig('maxcorr_violins.pdf',bbox_inches='tight')
# plt.show()

# fig=plt.figure(figsize = (8,10))
# ax = plt.axes()
# im=sns.boxplot(data = [TF_df['max_corr'].values,RBP_filt['max_corr'].values,prot_filt['max_corr'].values],ax=ax,showfliers=False)
# im.set_xticklabels(['TFs','RBPs (-TFs)','Proteome (-TFs)'],fontsize=15)
# im.set_ylabel("Max Cross-Correlation", fontsize = 15)
# ax.grid(False)
# ax.set_facecolor('white')
# ax.patch.set_edgecolor('black')  
# ax.patch.set_linewidth('2') 
# ax.figure.savefig('maxcorr_boxplots.pdf',bbox_inches='tight')
# plt.show()

# fig=plt.figure(figsize = (8,10))
# ax = plt.axes()
# im=sns.stripplot(data = [TF_df['max_corr'].values,RBP_filt['max_corr'].values,prot_filt['max_corr'].values],ax=ax)
# im.set_xticklabels(['TFs','RBPs (-TFs)','Proteome (-TFs)'],fontsize=15)
# im.set_ylabel("Max Cross-Correlation", fontsize = 15)
# ax.grid(False)
# ax.set_facecolor('white')
# ax.patch.set_edgecolor('black')  
# ax.patch.set_linewidth('2') 
# ax.figure.savefig('maxcorr_stripplots.pdf',bbox_inches='tight')
# plt.show()

In [9]:
# # https://medium.com/analytics-vidhya/love-the-ocean-love-seaborn-2e8737bef728

# df = pd.concat([TF_df,RBP_filt,prot_filt],keys=['TFs', 'RBPs (-TFs)','Proteome (-TFs)']).reset_index()

# sns.set(style="white", rc={"axes.facecolor": (0, 0, 0, 0)})

# # Initialize the FacetGrid object
# n=5 # Number of ridges to plot
# pal = sns.cubehelix_palette(n, rot=-.25, light=.7)
# g = sns.FacetGrid(df, row="level_0", hue="level_0", 
#                   aspect=10, height=1.0, palette=pal)

# # Draw the densities in a few steps
# g.map(sns.kdeplot, "max_corr", clip_on=False, shade=True, alpha=1, lw=1.5, bw=.2)
# g.map(plt.axhline, y=0, lw=2, clip_on=False)

# # Define and use a simple function to label the plot in axes coordinates
# def label(x, color, label):
#     ax = plt.gca()
#     ax.text(0, .2, label, fontweight="bold", color=color,
#             ha="left", va="center", transform=ax.transAxes)
# g.map(label, "max_corr")

# # Set the subplots to overlap
# g.fig.subplots_adjust(hspace=-.55)

# # Remove axes details that don't play well with overlap
# g.set_titles("")
# g.set(yticks=[])
# g.despine(bottom=True, left=True)

# g.savefig('maxcorr_ridgeplots.pdf',bbox_inches='tight')